In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import sweetviz as sv
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

In [ ]:
pd.__version__

In [ ]:
IMPORT_PATH = os.path.join("..", "input")
TEMP_PATH = os.path.join("..", "temp")
OUTPUT_PATH = os.path.join("..", "output")

In [ ]:
# Laden aller Dateien im .csv Format inkl. encoding und Ersetzen von "--" zu NaN
def load_data(file_name):
    file_path = os.path.join(IMPORT_PATH, file_name)
    df = pd.read_csv(file_path, delimiter=',',  
                     quotechar='"',             
                     encoding='utf-8',          
                     na_values='--',            
                     dtype=str)                 
    return df

## Import data

In [ ]:
ads_per_day_df = load_data("Report_Ads-per-day.csv")
print(ads_per_day_df.head())
print(ads_per_day_df.shape)
print(ads_per_day_df.columns)

In [ ]:
hour_of_the_day_df = load_data("Report_Hour-of-the-day.csv")
print(hour_of_the_day_df.head())
print(hour_of_the_day_df.shape)
print(hour_of_the_day_df.columns)

In [ ]:
asset_details_df = load_data("Report_AdGroup_AssetDetails.csv")
print(asset_details_df.head())
print(asset_details_df.shape)
print(asset_details_df.columns)

In [ ]:
search_keyword_df = load_data("Report_SearchKeyword.csv")
print(search_keyword_df.head())
print(search_keyword_df.shape)
print(search_keyword_df.columns)

## Inspect

In [ ]:
#Profiling report via Sweetviz 
ads_per_day_viz = sv.analyze(ads_per_day_df)
ads_per_day_viz.show_html(os.path.join(OUTPUT_PATH, 'ads_per_day_viz.html'))

hour_of_the_day_viz = sv.analyze(hour_of_the_day_df)
hour_of_the_day_viz.show_html(os.path.join(OUTPUT_PATH, 'hour_of_the_day_viz.html'))

asset_details_viz = sv.analyze(asset_details_df)
asset_details_viz.show_html(os.path.join(OUTPUT_PATH, 'asset_details_viz.html'))

search_keyword_viz = sv.analyze(search_keyword_df)
search_keyword_viz.show_html(os.path.join(OUTPUT_PATH, 'search_keyword_viz.html'))

## Transform

### Data cleansing

In [ ]:
# a) Anpassung der NaN values im 'search_keyword_df'
search_keyword_df['Quality Score'] = search_keyword_df['Quality Score'].fillna('0')
search_keyword_df['Avg. engagement duration per session (seconds) (GA4)'] = search_keyword_df['Avg. engagement duration per session (seconds) (GA4)'].fillna('0')
search_keyword_df['% Engaged sessions (GA4)'] = search_keyword_df['% Engaged sessions (GA4)'].fillna('0%')

# Überprüfung der Änderungen
print(search_keyword_df.head())
print(search_keyword_df.isna().sum())  # Überprüfen, ob alle NaN-Werte entfernt wurden

In [ ]:
#Prüfen der Anpassung
search_keyword_viz_fin = sv.analyze(search_keyword_df)
search_keyword_viz_fin.show_html(os.path.join(OUTPUT_PATH, 'search_keyword_viz_fin.html'))

In [ ]:
# b) Anpassung der NaN values im 'hour_of_the_day_df'
cols_cpc = [col for col in hour_of_the_day_df.columns if 'Avg. CPC' in col]
hour_of_the_day_df[cols_cpc] = hour_of_the_day_df[cols_cpc].apply(lambda x: x.fillna('0'))
cols_ctr_interaction = [col for col in hour_of_the_day_df.columns if 'CTR' in col or 'Interaction rate' in col]
hour_of_the_day_df[cols_ctr_interaction] = hour_of_the_day_df[cols_ctr_interaction].apply(lambda x: x.fillna('0%'))

print(hour_of_the_day_df.head(20))

In [ ]:
#Prüfen der Anpassung
hour_of_the_day_viz_fin = sv.analyze(hour_of_the_day_df)
hour_of_the_day_viz_fin.show_html(os.path.join(OUTPUT_PATH, 'hour_of_the_day_viz_fin.html'))

In [ ]:
# Schmelzen des hour_of_the_day_df DataFrames, um die Tages- und Stundenwerte zu kombinieren
hour_of_the_day_melted = hour_of_the_day_df.melt(id_vars=['Ad group', 'Ad group ID', 'Hour of the day'], 
                                                 var_name='Day_Metric', 
                                                 value_name='Value')

# Extrahieren von Tag und Metrik aus der "Day_Metric" Spalte
hour_of_the_day_melted[['Day_of_week', 'Metric']] = hour_of_the_day_melted['Day_Metric'].str.extract(r'(\w+)_(.*)')

# Umbenennen der Spalten nach dem Melting und vor dem Pivoting
hour_of_the_day_melted['Metric'] = 'Hour_' + hour_of_the_day_melted['Metric']

# Pivoting des DataFrames, um die ursprüngliche Struktur zu erhalten, aber aufgeschlüsselt nach Tag und Stunde
hour_of_the_day_pivot = hour_of_the_day_melted.pivot_table(index=['Ad group', 'Ad group ID', 'Hour of the day', 'Day_of_week'], 
                                                           columns='Metric', 
                                                           values='Value', 
                                                           aggfunc='first').reset_index()

# Umbenennen der Spalten nach dem Pivoting
hour_of_the_day_pivot.columns = ['_'.join(col).strip() if type(col) is tuple else col for col in hour_of_the_day_pivot.columns]
hour_of_the_day_pivot.rename(columns={'Ad group': 'Ad_group', 'Ad group ID': 'Ad_group_ID', 'Hour of the day': 'Hour_of_day'}, inplace=True)

# Anzeigen der ersten Zeilen des umstrukturierten DataFrames
print(hour_of_the_day_pivot.head())
print(hour_of_the_day_pivot.columns)

## Merging

In [ ]:
# Merge von ads_per_day_df und hour_of_the_day reports
# Sicherstellen, dass die Spaltennamen in beiden DataFrames konsistent sind
ads_per_day_df.rename(columns={'Ad group': 'Ad_group', 'Ad group ID': 'Ad_group_ID', 'Day of the week': 'Day_of_week'}, inplace=True)
hour_of_the_day_pivot.rename(columns={'Ad group': 'Ad_group', 'Ad group ID': 'Ad_group_ID', 'Day': 'Day_of_week'}, inplace=True)

# Mergen der beiden DataFrames basierend auf Ad_group_ID und Day_of_week
merged_df = pd.merge(ads_per_day_df, hour_of_the_day_pivot, on=['Ad_group_ID', 'Day_of_week'], how='inner')

# Umbenennen der Spalten mit dem Suffix `_y`
merged_df.rename(columns={
    'Ad_group_y': 'Hour_Ad_group',
    'Clicks_y': 'Hour_Clicks',
    'CTR_y': 'Hour_CTR',
    'Cost_y': 'Hour_Cost',
    'Impr': 'Hour_Impr',
    'Interaction': 'Hour_Interaction'
}, inplace=True)

# Anzeigen der ersten Zeilen des gemergten DataFrames
print(merged_df.head())
print(merged_df.columns)

In [ ]:
# 3. Merge: search_keyword_df und merged_df
# Umbenennen der Spalten, um Konflikte zu vermeiden
search_keyword_df.rename(columns={
    'Ad group': 'Ad_group',
    'Ad group ID': 'Ad_group_ID',
    'Impr.': 'Keyword_Impr',
    'Clicks': 'Keyword_Clicks',
    'CTR': 'Keyword_CTR',
    'Cost': 'Keyword_Cost',
    'Avg. CPC': 'Keyword_Avg_CPC',
    '% Engaged sessions (GA4)': 'Keyword_%-Engaged_sessions',
    'Avg. engagement duration per session (seconds) (GA4)': 'Keyword_Engagement_seconds'
}, inplace=True)

# Auswahl der relevanten Spalten aus search_keyword_df
search_keyword_df = search_keyword_df[['Ad_group_ID', 'Search keyword', 'Keyword_Impr', 'Keyword_Clicks', 'Keyword_CTR', 'Keyword_Cost', 'Keyword_Avg_CPC', 'Keyword_%-Engaged_sessions', 'Keyword_Engagement_seconds']]

# Mergen der DataFrames basierend auf Ad_group_ID
merged_df = pd.merge(merged_df, search_keyword_df, on='Ad_group_ID', how='inner')

# Anzeigen der ersten Zeilen des gemergten DataFrames
print(merged_df.head())
print(merged_df.columns)

In [ ]:
# 4. Merge: asset_details_df und merged_df
# Umbenennen der Spalten, um Konflikte zu vermeiden
asset_details_df.rename(columns={
    'Ad group': 'Ad_group',
    'Ad group ID': 'Ad_group_ID',
    'Impr.': 'Asset_Impr'
}, inplace=True)

# Auswahl der relevanten Spalten aus asset_details_df
asset_details_df = asset_details_df[['Ad_group_ID', 'Asset', 'Asset type', 'Asset_Impr']]

# Mergen der DataFrames basierend auf Ad_group_ID
merged_df = pd.merge(merged_df, asset_details_df, on='Ad_group_ID', how='inner')

# Anzeigen der ersten Zeilen des final gemergten DataFrames
print(merged_df.head(10))
print(merged_df.columns)

In [ ]:
#Entfernen der Spalten "Ad_group_y" und "Mobile final URL"
merged_df.drop(columns=['Hour_Ad_group'], inplace=True)

In [ ]:
print(merged_df.head(10))
print(merged_df.columns)

In [ ]:
# Imputation der fehlenden Werte mit "0" und "0%"
columns_to_impute_numeric = ['Avg. engagement duration per session (seconds) (GA4)', 'Events / session (GA4)']
column_to_impute_percent = '% Engaged sessions (GA4)'

# Auffüllen der numerischen Spalten mit "0"
for column in columns_to_impute_numeric:
    merged_df[column] = merged_df[column].fillna(0)

# Auffüllen der prozentualen Spalte mit "0%"
merged_df[column_to_impute_percent] = merged_df[column_to_impute_percent].fillna("0%")

# Entfernen des Prozentzeichens aus den relevanten Spalten, ohne die Werte zu konvertieren
percent_columns = [
    'Hour_CTR', 'Hour_Interaction rate', 'CTR', 
    'Keyword_CTR', 'Keyword_%-Engaged_sessions'
]

# Funktion zum Entfernen des Prozentzeichens
def remove_percent(column):
    return column.str.rstrip('%')

# Anwenden der Funktion auf die ausgewählten Spalten
for col in percent_columns:
    merged_df[col] = remove_percent(merged_df[col])

# Überprüfen der Änderungen
print(merged_df[percent_columns].head())

# Überprüfen, ob alle fehlenden Werte aufgefüllt wurden
print(merged_df[columns_to_impute_numeric + [column_to_impute_percent]].isnull().sum())


In [ ]:
# Um in den Transformer pipelines auf Textspezifika zugreifen zu können, werden hier vor dem Train-test-split einen Word count erzeugen.
def count_words(text):
    if pd.isna(text):
        return 0
    return len(str(text).split())

# List of text columns to process
text_features = ['Asset', "Search keyword"]

# Apply the word count function to each text column and store the result in a new column
for feature in text_features:
    merged_df[f'{feature}_word_count'] = merged_df[feature].apply(count_words)

## Train-test-split & export

In [ ]:
#Aufteilung Traings- und Testdaten
train_df, test_df = train_test_split(merged_df, train_size=.8, random_state=42)

In [ ]:
#Speichern im .pickle Format
train_df.to_pickle(os.path.join(TEMP_PATH, "train.pickle"))
test_df.to_pickle(os.path.join(TEMP_PATH, "test.pickle"))

# EDA

## Import pickle

In [ ]:
train_df = pd.read_pickle(os.path.join(TEMP_PATH, "train.pickle"))
train_df.head()
print(train_df.columns)

## Analyse der Daten

In [ ]:
ges_train = sv.analyze(train_df, pairwise_analysis='off')
ges_train.show_html('ges_train.html')

## Auswahl der Variablen

In [ ]:
# Auswahl der numerischen und kategorialen Variablen
numerical_features = [
    'Clicks', 'Cost', 'Avg. CPC', 
    'Hour_Clicks', 'Hour_Cost', 'Hour_Impr.', 'Hour_Interaction rate',
    'Keyword_Impr', 'Keyword_Clicks', 'Keyword_Cost', 'Keyword_Avg_CPC',
    'Keyword_Engagement_seconds', 'Asset_Impr', 'Asset_word_count', 'Search keyword_word_count'
]

categorical_features = [
    'Search keyword', 'Asset', 'Asset type', 'Day_of_week'
]

# Zielvariable (Target)
target = 'CTR'

# Erstellen eines  DataFrames mit den ausgewählten Variablen
train_df = train_df[numerical_features + categorical_features + [target]]

# Überprüfen der ersten Zeilen des neuen DataFrames
print(train_df.head())

### Visualisierung statistischer Analysen

In [ ]:
# Setzen des Stils für die Plots
sns.set(style="whitegrid")

# Korrelationen zwischen den numerischen Variablen berechnen
corr = train_df[numerical_features].corr()

# Heatmap der Korrelationen erstellen
plt.figure(figsize=(12, 8))
sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Korrelation zwischen numerischen Variablen')
plt.show()

# Verteilung der numerischen Variablen plotten
train_df[numerical_features].astype(float).hist(bins=30, figsize=(20, 15))
plt.suptitle('Verteilung der numerischen Variablen')
plt.show()

# Paarplot der numerischen Variablen
sns.pairplot(train_df[numerical_features].astype(float))
plt.suptitle('Paarplot der numerischen Variablen')
plt.show()

# Boxplot der Zielvariablen im Verhältnis zu den numerischen Variablen
plt.figure(figsize=(12, 8))
for i, col in enumerate(numerical_features, 1):
    plt.subplot(4, 4, i)
    sns.boxplot(x=train_df[target], y=train_df[col].astype(float))
    plt.title(f'Boxplot von {col} im Verhältnis zu {target}')
plt.tight_layout()
plt.show()

# Zählplot für kategoriale Variablen
plt.figure(figsize=(20, 20))
num_plots = len(categorical_features)
cols = 4
rows = (num_plots // cols) + (num_plots % cols > 0)

for i, col in enumerate(categorical_features, 1):
    plt.subplot(rows, cols, i)
    sns.countplot(y=train_df[col])
    plt.title(f'Zählplot von {col}')
plt.tight_layout()
plt.show()
